In [1]:
import praw
import os
import sqlite3

In [2]:
REDDIT_CLIENT_ID=os.environ["REDDIT_CLIENT_ID"]
REDDIT_CLIENT_SECRET=os.environ["REDDIT_CLIENT_SECRET"]
REDDIT_USER_AGENT=os.environ["REDDIT_USER_AGENT"]

In [3]:
def extract():
    client = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secrect=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT,
    )

    subreddit = client.subreddit('dataengineering')
    top_subreddit = subreddit.hot(limit=100)
    data = []

    for submission in top_subreddit:
        data.append(
            {
                'title': submission.title,
                'score': submission.score,
                'id': submission.id,
                'url': submission.url, 
                'comments': submission.num_comments,
                'created': submission.created,
                'text': submission.selftext,
            }
        )
    return data

In [4]:
def transform(data):
    """
    Function to only keep outliers.
    Outliers are based on num of comments > 2 standard deviations from mean
    """
    num_comments = [post.get('comments') for post in data]
    mean_num_comments = sum(num_comments) / len(num_comments)

    std_num_comments = (sum([(x - mean_num_comments) ** 2 for x in num_comments]) / len(num_comments)) ** 0.5

    return [
        post
        for post in data
        if post.get('comments') > mean_num_comments + 2 * std_num_comments
    ]

In [6]:
def load(data):
    #Create a database connection
    conn = sqlite3.connect('./data/socialetl.db')
    cur = conn.cursor()
    try:
        #Insert data into database
        for post in data:
            cur.execute(
                """
                    INSERT INTO social_posts (
                        id, source, social_data
                    ) VALUES (
                        :id, :source, :social_data
                    )
                    """,
                {
                    'id': post.get('id'),
                    'score': post.get('score'),
                    'social_data': str(
                        {
                            'title': post.get('title'),
                            'url': post.get('url'),
                            'comments': post.get('num_comments'),
                            'created': post.get('created'),
                            'text': post.get('selftext')
                        }
                    )
                }
            )
    finally:
        conn.commit()
        conn.close()

In [7]:
def main():
    #Pull data from reddit
    data = extract()
    #Transform reddit data
    transformed_data = transform(data)
    #Load data into database
    load(transformed_data)